<a href="https://colab.research.google.com/github/juanjosemarinm/SyS2025-2/blob/main/TAREA_triangulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

T = 4 # intervalo de la señal
B = 5 #altura de la señal
d1 = 1 # d1 < T/2
Fs = 200*1/T # frecuencia muestreo
Ts = 1/Fs # periodo de muestreo
t = np.arange(-T/2,T/2,Ts)
x = np.zeros(len(t))
x[(t>=-d1) & (t<0)] = (B/d1)*t[(t>=-d1) & (t<0)]+B
x[(t>=0) & (t<d1)] = -(B/d1)*t[(t>=0) & (t<d1)]+B



In [ ]:
import matplotlib.pyplot as plt
plt.plot(t,x)
plt.xlabel('t')
plt.ylabel('x(t)')
plt.grid()
plt.show()

In [ ]:
N = 25 #numero armonicos
wo = 2*np.pi/T # frecuencia fundamental
# definir bases
Nm = len(t)
phin = np.zeros((Nm,2*N+1),dtype=np.complex128)#crear matriz para guardar bases
for n in range(-N,N+1,1):
    phin[:,n+N] = np.exp(1j*n*wo*t)#base de fourier en el intervalo de interés

In [ ]:


plt.plot(t,phin)
plt.show()

$$c_n = \frac{B}{\pi n^2 \omega_o d_1}\left( 1 - \cos(n\omega_o d_1)\right)$$

In [ ]:
#calcular espectro -> cambiar según la señal estudiada
cn = np.zeros(2*N+1,dtype=np.complex128)
nv = np.linspace(-N, N, 2*N+1) #vector num armónicos

# Calculate the DC component of x
dc_component_x = np.mean(x)

# Calculate Cn for non-zero nv
non_zero_nv_indices = nv != 0
Cn = np.zeros(2*N+1,dtype=np.complex128)
Cn[non_zero_nv_indices] = (B/(wo*d1*np.pi*nv[non_zero_nv_indices]**2))*(1-np.cos(nv[non_zero_nv_indices]*wo*d1))

# Set the DC component of the Fourier coefficients
Cn[N] = dc_component_x # nivel dc C_0

In [ ]:
#graficar espectro
fig = plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.stem(nv, np.real(Cn), 'r')
plt.xlabel(r'$n$',fontsize = 14)
plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,2)
plt.stem(nv, np.imag(Cn), 'r')
plt.xlabel(r'$n$',fontsize = 14)
plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,3)
plt.stem(nv, abs(Cn), 'r')#magnitud del espectro
plt.xlabel(r'$n$',fontsize = 14)
plt.ylabel(r'$|C_n|$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,4)
plt.stem(nv, np.angle(Cn), 'r')#fase del espectro
plt.xlabel(r'$n$',fontsize = 14)
plt.ylabel(r'$\langle C_n$',fontsize = 14)
plt.axis('tight')
plt.grid()

fig.tight_layout()

In [ ]:
xe = phin.dot(Cn) # dot calcula el producto entre vectores
plt.plot(t,xe)
plt.show()

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

Px = (2/3) * (B**2 * d1) / T # estimar según señal estudiada
def pltest(Na=1):
    ind = np.arange(N-Na,N+Na+1)
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    #señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
    plt.plot(t,xe,color='b',label='$x_e(t)$') # señal estimada o filtrada
    plt.plot(t,x,color='r',label='x(t)') #señal original
    plt.title('$E_r$=%.2f[%%]' % (100*er),fontsize=16)
    plt.xlabel('t[s]')
    plt.ylabel('x(t)')
    plt.grid()
    plt.legend()
    plt.show()
    return
interact(pltest,Na=(1,N,1))

In [ ]:
from matplotlib import animation
from matplotlib import rc

#figura y axes animaciónx
fig, ax = plt.subplots(2, 3,figsize=(12,5))
#función para gráfico interactivo
def update(n):
    #actualización regresor Bayesiano
    #Graficar prior
    ax[0,0].clear()
    ax[0,1].clear()
    ax[0,2].clear()
    ax[1,0].clear()
    ax[1,1].clear()
    ax[1,2].clear()

    #espectro
    ind = np.arange(N-n,N+n+1) #armónicos a considerar
    stem1 = ax[0,0].stem(nv, abs(Cn), 'b')#magnitud del espectro
    stem2 = ax[0,0].stem(nv[ind], abs(Cn[ind]), 'r')#magnitud del espectro
    ax[0,0].set_xlabel(r'$n$[rad/s]')
    ax[0,0].set_ylabel(r'$|C_n|$')

    #componentes espectrales
    ce = phin[:,ind].dot(np.diag(Cn[ind]) )
    line1 = ax[0,1].plot(t,np.real(ce))#componente real
    ax[0,1].set_xlabel('t[s]')
    ax[0,1].set_ylabel('$Re\{c_n\phi_n(t)\}$')

    line2 = ax[0,2].plot(t,np.imag(ce))#componente imaginario
    ax[0,2].set_xlabel('t[s]')
    ax[0,2].set_ylabel('$Im\{c_n\phi_n(t)\}$')




    #reconstruccion
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    #señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
    #error en t
    et = x - xe

    line3 = ax[1,1].plot(t,x,'r',linewidth=4,label='$x(t)$')
    line4 = ax[1,1].plot(t,xe,color='b',label='$x_e(t)$') # señal estimada o filtrada
    line5 = ax[1,1].plot(t,et,color='g',label='$e(t)$') # señal estimada o filtrada
    ax[1,1].set_title('$E_r$=%.2f[%%]' % (100*er))
    ax[1,1].set_xlabel('t[s]')
    ax[1,1].set_ylabel('x(t)')
    ax[1,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax[1,0].axis('off')
    ax[1,2].axis('off')
    plt.subplots_adjust(wspace=0.5,hspace=0.7)

    return stem1, stem2, line1[0], line2[0], line3[0], line4[0], line5[0]

#desplegar animación
anim = animation.FuncAnimation(fig, update, frames=np.arange(0,N+1,1), blit=False)
rc('animation', html='jshtml')

In [ ]:
anim